In [1]:
import requests
from datetime import datetime
import os
from dotenv import load_dotenv
import xml.etree.ElementTree as ET

In [55]:
def get_lunar_date(solar_date, base_url, service_key):
    # solYear, solMonth, solDay 추출
    solYear = solar_date.year
    solMonth = f"{solar_date.month:02d}"  # 두 자리로 포맷
    solDay = f"{solar_date.day:02d}"      # 두 자리로 포맷

    # API 요청
    response = requests.get(f"{base_url}?solYear={solYear}&solMonth={solMonth}&solDay={solDay}&ServiceKey={service_key}")
    
    # 응답 상태 코드 체크
    if response.status_code != 200:
        raise Exception(f"Error: 오류가 발생했습니다. status code {response.status_code}")
    
    #디버깅용
    print("Response content (decoded):", response.content.decode('utf-8'))
    
    try:
        # XML 응답을 파싱
        root = ET.fromstring(response.content)
    except ET.ParseError:
        raise Exception("Error: XML 파싱 오류가 발생했습니다.")
    
    # 헤더 파싱
    header = {
        "resultCode": root.findtext('.//header/resultCode'),
        "resultMsg": root.findtext('.//header/resultMsg')
    }
    
    # body의 item 정보 파싱
    items = []
    for item in root.findall('.//item'):
        lun_year = item.findtext('lunYear')
        lun_month = item.findtext('lunMonth')
        lun_day = item.findtext('lunDay')
        lun_iljin = item.findtext('lunIljin')
        lun_leapmonth = item.findtext('lunLeapmonth')
        lun_nday = item.findtext('lunNday')
        lun_secha = item.findtext('lunSecha')
        lun_wolgeon = item.findtext('lunWolgeon')
        sol_year = item.findtext('solYear')
        sol_month = item.findtext('solMonth')
        sol_day = item.findtext('solDay')
        sol_jd = item.findtext('solJd')
        sol_leapyear = item.findtext('solLeapyear')
        sol_week = item.findtext('solWeek')
        
        # 아이템의 모든 값을 딕셔너리로 저장
        items.append({
            "lun_year": lun_year,
            "lun_month": lun_month,
            "lun_day": lun_day,
            "lun_iljin": lun_iljin,
            "lun_leapmonth": lun_leapmonth,
            "lun_nday": lun_nday,
            "lun_secha": lun_secha,
            "lun_wolgeon": lun_wolgeon,
            "sol_year": sol_year,
            "sol_month": sol_month,
            "sol_day": sol_day,
            "sol_jd": sol_jd,
            "sol_leapyear": sol_leapyear,
            "sol_week": sol_week
        })

    return {
        "header": header,
        "items": items,
    }

In [34]:
# 변수 지정
base_url = "http://apis.data.go.kr/B090041/openapi/service/LrsrCldInfoService/getLunCalInfo"
heavenly_stems = ["갑", "을", "병", "정", "무", "기", "경", "신", "임", "계"]
earthly_branches = ["자", "축", "인", "묘", "진", "사", "오", "미", "신", "유", "술", "해"]

In [78]:
# api_key 가져오기
load_dotenv('../env/data.env')  # .env 파일에서 환경 변수 로드
service_key = os.getenv('API_KEY')  # 환경 변수에서 서비스 키 가져오기

print(service_key)

Lvn%2FX9ciaH3OcErj46QABbDpndkMA%2FBR6ZJmLMlTOO1No1vGocwgMhcp%2BVKl%2BShi8et1lD%2BVhhVAdQNi%2BtkKGw%3D%3D


In [79]:
# 양력일자를 solYear, solMonth, solDay 형태로 입력받기
birth_date = datetime(1995, 10, 10)  # 예: 1990년 5월 15일
birth_hour = 14  # 예: 14시 (오후 2시)
    
saju = get_lunar_date(birth_date, base_url, service_key)

# 결과값 확인용
print(saju)

Response content (decoded): <?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><lunDay>16</lunDay><lunIljin>갑술(甲戌)</lunIljin><lunLeapmonth>윤</lunLeapmonth><lunMonth>08</lunMonth><lunNday>29</lunNday><lunSecha>을해(乙亥)</lunSecha><lunWolgeon></lunWolgeon><lunYear>1995</lunYear><solDay>10</solDay><solJd>2450001</solJd><solLeapyear>평</solLeapyear><solMonth>10</solMonth><solWeek>화</solWeek><solYear>1995</solYear></item></items><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>1</totalCount></body></response>
{'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'items': [{'lun_year': '1995', 'lun_month': '08', 'lun_day': '16', 'lun_iljin': '갑술(甲戌)', 'lun_leapmonth': '윤', 'lun_nday': '29', 'lun_secha': '을해(乙亥)', 'lun_wolgeon': '', 'sol_year': '1995', 'sol_month': '10', 'sol_day': '10', 'sol_jd': '2450001', 'sol_leapyear': '평', 'sol_week': '화'}]}


In [ ]:
# 파싱한 데이터 표현 정리 필요